In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### XToYHTo4B EOY production

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
                                                     pythia8CommonSettingsBlock,
                                                     pythia8CP5SettingsBlock,
                                                     pythia8PSweightsSettingsBlock,
                                                     processParameters = cms.vstring(
                                                     __CHANNEL_DECAY_FRAGMENT__
                                                     ),
                                                     parameterSets = cms.vstring('pythia8CommonSettings',
                                                                                 'pythia8CP5Settings',
                                                                                 'pythia8PSweightsSettings',
                                                                                 'processParameters'
                                                                                 )
                                                     )
                         )


ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator (decay_fr):
    dict = { '2016': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2016","2017", "2018"]
#mX = ["280","320","360","1600","1800","2000","2200", "2400", "2500", "2600", "2800", "3000", "3500", "4000"]
#mY = ["60", "70", "80", "90", "100", "125", "150", "200", "250", "300"]
#mY2 = ["60", "70", "80", "90", "100", "125", "150", "170", "190", "250", "300", "350", "400", "450", "500", "550", "600", "650", "700", "800", "900", "1000", "1100", "1200", "1300", "1400", "1600", "1800", "2000", "2200", "2400", "2500", "2600", "2800"]
#mY3 = ["650", "700", "800", "900", "1000", "1100", "1200", "1300", "1400", "1600", "1800", "2000", "2200", "2400", "2500", "2600", "2800"]

###Addition
mX = ["650"]
mY = ["60", "70", "80", "90", "100", "125", "150", "170", "190", "200", "250", "300", "350", "400", "450", "500"]

tot_events1 = [400000]*len(mY)
#tot_events2 = [400000]*len(mY2)
#tot_events3 = [400000]*len(mY3)
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    print(year)
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/pre2017/13TeV/NMSSM_XToYH'
    t_datasetname_year = 'NMSSM_XToYHTo4B_MX-{massX}_MY-{massY}_TuneCP5_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/pre2017/13TeV/madgraph/V5_2.4.2/NMSSM_XToYH_MX_{massX}_MY_{massY}/v1/NMSSM_XToYH_MX_{massX}_MY_{massY}_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    
    dataset_names_year = []
    gp_ggf_year = []

    for mx in mX:
        #if int(mx) <= 360:
        for my in mY:
            d = {"massX": mx, "massY": my}
            tmp_example_dict = t_example1
            gp_ggf_year.append(t_gp_year.format_map(d))
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict = dataset_names_year
            tmp_gridpack_dict = gp_ggf_year
        '''        
        elif int(mx) <= 2000 and int(mx) >= 1600 and year == 0:
            for my in mY3:
                d = {"massX": mx, "massY": my}
                tmp_example_dict = t_example1
                gp_ggf_year.append(t_gp_year.format_map(d))
                dataset_names_year.append(t_datasetname_year.format_map(d))
                tmp_dataset_dict = dataset_names_year
                tmp_gridpack_dict = gp_ggf_year
        elif int(mx) >= 2200:
            for my in mY2:
                d = {"massX": mx, "massY": my}
                tmp_example_dict = t_example1
                gp_ggf_year.append(t_gp_year.format_map(d))
                dataset_names_year.append(t_datasetname_year.format_map(d))
                tmp_dataset_dict = dataset_names_year
                tmp_gridpack_dict = gp_ggf_year
        '''
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

#print(dataset_names['2017'])
#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

In [ ]:
x_hy_bbbb = '''                                                                                                                                                                                           
    '25:onMode = off', # Turn off all H decays
    '25:oneChannel = 1 1 100 5 -5', # H-> b b
    '25:onIfMatch = 5 -5',
    '35:onMode = off',
    '35:oneChannel = 1 1 100 5 -5',  # Y-> b b
    '35:onIfMatch = 5 -5',
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
    'ResonanceDecayFilter:mothers = 25,35', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
    'ResonanceDecayFilter:daughters = 5,5,5,5',
'''
#'SpaceShower:dipoleRecoil = on' 


## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator(x_hy_bbbb)
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    with open('XtoYHto4B' + '_additional_EOY_' + year +'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])

        k=0
        for i in range(len(mX)):
            #if int(mX[i]) <= 360:
            for j in range(len(mY)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.4.2"
                dataset_name = dataset_names[year][k]
                if year == "2016":
                    events = tot_events1[j]
                    #prepid = 'HIG-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                elif year == "2017":
                    events = tot_events1[j]
                    #prepid = 'HIG-RunIISummer20UL17wmLHEGEN-0' + str(pr)
                elif year == "2018":
                    events = tot_events1[j]
                    #prepid = 'HIG-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                    
                tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
                final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
                note = dataset_name.replace('_',' ')
                generators="Madgraph_" + version + "  Pythia8"
                mcdb_id = '0'
                time = '33'
                size = '1000'
                if int(mX[i]) > (int(mY[j]) + 125): 
                    csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                #pr = pr+1   
                k=k+1
            '''        
            elif int(mX[i]) <= 2000 and int(mX[i]) >= 1600 and year == "2016":
                for j in range(len(mY3)):
                    tmp_fragment=""
                    #print(gridpacks_dict[year][i])
                    #print(dataset_names[year][i])
                    #version = MadgraphVersion.search(path).group(0)
                    version="2.4.2"
                    dataset_name = dataset_names[year][k]
                    if year == "2016":
                        events = tot_events3[j]
                        #prepid = 'HIG-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                    elif year == "2017":
                        print("nooo")
                        events = tot_events3[j]
                        #prepid = 'HIG-RunIISummer20UL17wmLHEGEN-0' + str(pr)
                    elif year == "2018":
                        print("nooo")
                        events = tot_events3[j]
                        #prepid = 'HIG-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                    
                    tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
                    final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
                    note = dataset_name.replace('_',' ')
                    generators="Madgraph_" + version + "  Pythia8"
                    mcdb_id = '0'
                    time = '62'
                    size = '925'
                    if int(mX[i]) > (int(mY3[j]) + 125): 
                        csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                    #pr = pr+1   
                    k=k+1
            elif int(mX[i]) >= 2200:
                for j in range(len(mY2)):
                    tmp_fragment=""
                    #print(gridpacks_dict[year][i])
                    #print(dataset_names[year][i])
                    #version = MadgraphVersion.search(path).group(0)
                    version="2.4.2"
                    dataset_name = dataset_names[year][k]
                    if year == "2016":
                        events = tot_events2[j]
                        #prepid = 'HIG-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                    elif year == "2017":
                        events = tot_events2[j]
                        #prepid = 'HIG-RunIISummer20UL17wmLHEGEN-0' + str(pr)
                    elif year == "2018":
                        events = tot_events2[j]
                        #prepid = 'HIG-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                    
                    tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
                    final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
                    note = dataset_name.replace('_',' ')
                    generators="Madgraph_" + version + "  Pythia8"
                    mcdb_id = '0'
                    time = '62'
                    size = '925'
                    if int(mX[i]) > (int(mY2[j]) + 125): 
                        csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                    k=k+1
                    #pr = pr+1
            '''